In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [52]:
#Uploading the data
xls = pd.ExcelFile('The_data.xlsx')

#Age and Sex
df_age_sex = pd.read_excel(xls, 'Age and Sex')
df_age_sex = df_age_sex.drop('YEAR', 1)
df_age_sex.columns = ["Sra", "AGE GROUP", "TOTAL", "MALE", "FEMALE"]


#elevation
df_elevation = pd.read_excel(xls, 'Elevation')
df_elevation['Elevation']=df_elevation['Elevation'].fillna("No Data")


#Marital Status
df_marital_status = pd.read_excel(xls, 'Marital Status')
df_marital_status = df_marital_status.drop('YEAR', 1)


#Educational Attainment
df_educational_attainment = pd.read_excel(xls, 'Educational Attainment')
df_educational_attainment = df_educational_attainment.drop('YEAR', 1)


#Types of Households
df_housing_units_type = pd.read_excel(xls, 'Housing Units Type')
df_housing_units_type = df_housing_units_type.drop('YEAR', 1)


#House Value
df_housing_value = pd.read_excel(xls, 'Housing Value')
df_housing_value = df_housing_value.drop('YEAR', 1)


#House Structure Age
df_year_house_built = pd.read_excel(xls, 'Year Structure Built')
df_year_house_built = df_year_house_built.drop('YEAR', 1)


#Vehicle Data
df_vehicles = pd.read_excel(xls, 'Vehicle Availability')
df_vehicles = df_vehicles.drop('YEAR', 1)

#Population Density
df_pop_density = pd.read_excel(xls, 'Population Density')



In [53]:
#Making a dataframe... with the data..




In [54]:
#The unique regions (SRAs) we have for the census data

regions = df_age_sex["Sra"].unique()
df_regions = pd.DataFrame(regions)
df_regions.columns = ["Region"]

In [55]:
#################################################################
# Our main dataframe that we'll use to create the SIR equations #
#################################################################

df = pd.DataFrame(data=regions, columns=["Region"])

In [56]:
#Adding median age to the df

median_age = df_age_sex[(len(df_age_sex) - 41):]
median_age = median_age["TOTAL"]
median_age.index=df.index

df['Median Age'] = (median_age)

In [57]:
#Adding house median value to the df

median_house_value = df_housing_value[(len(df_housing_value) - 41):]
median_house_value = median_house_value["NUMBER"]
median_house_value.index=df.index

df['Median House Value'] = (median_house_value)

In [58]:
#Population density...

df_pop_density = df_pop_density.sort_values("Neighborhood")

df['Population Density'] = ("n/a")
missing_data_df = pd.DataFrame([['Alpine', 569],['Anza-Borrego Springs',0],['Central San Diego',0],['Coastal',0],['Del Mar-Mira Mesa',6992],['Elliott-Navajo',6364],
                                ['Harbison-Crest',886],['Kearny Mesa',625],['Laguna-Pine Valley',24],['Mid-City',11237],['Miramar',397],['Mountain Empire',20],
                                ['North San Diego',0],['Palomar-Julian',11],['Pauma',82],['Peninsula',4696],['San Dieguito',0],['South Bay',0],
                                ['Southeastern San Diego',17908], ['Sweetwater',3362],['University',0],['Valley Center',833]], columns = ['Neighborhood', 'People per square mile'])
df_pop_density = pd.concat([df_pop_density, missing_data_df])
df_pop_density




,Neighborhood,People per square mile
23,Bonita,2710
36,Borrego Springs,60
1,Bostonia,7930
34,Boulevard,80
22,Carlsbad,2840
20,Casa de Oro-Mt Helix,2870
8,Chula Vista,5000
21,Coronado,2860
26,Del Mar,2460
35,Descanso,60


In [59]:
pd.options.mode.chained_assignment = None  # default='warn'


common_sras = []
common_sras_indices = []

for index, row in df.iterrows():
    if(df_pop_density['Neighborhood'].str.contains(row["Region"]).any()):
        common_sras.append(row["Region"])
        common_sras_indices.append(index)

for i in range(0,len(common_sras)):
    df["Population Density"][common_sras_indices[i]] = df_pop_density[df_pop_density["Neighborhood"] == common_sras[i]]
    x = df["Population Density"][common_sras_indices[i]]
    df["Population Density"][common_sras_indices[i]] = int(x["People per square mile"])

        

In [60]:
#Calculating the total population...

total_population = [0]*(len(regions))

k = 0
j = 0
for i in range(0, len(regions)): # i is for regions[]
    if(df_age_sex["Sra"][k] == regions[i]):
        j = k
        k = k + 20
        if(df_age_sex["Sra"][k] == "Mountain Empire"):
            #Because Miramar's data is wonky
            k = k - 4
        
        while(df_age_sex["Sra"][j] == regions[i]):
            total_population[i] += int(df_age_sex["TOTAL"][j])
            j += 1
            
#Add the Total Population column
df['Total Population'] = total_population

In [61]:
df

,Region,Median Age,Median House Value,Population Density,Total Population
0,Alpine,42.3,612870,569,15256
1,Anza-Borrego Springs,55.3,306852,0,4436
2,Carlsbad,40.8,613993,2840,114712
3,Central San Diego,34.4,521767,0,165582
4,Chula Vista,33.9,359311,5000,115234
5,Coastal,35.4,969973,0,75375
6,Coronado,31.3,1275412,2860,24697
7,Del Mar-Mira Mesa,37.0,639308,6992,158106
8,El Cajon,34.8,421170,6950,124552
9,Elliott-Navajo,37.4,516103,6364,89106
